In [14]:
from lea_utils import SimulatorLEA
from lea_utils import norm_values
import numpy as np

In [15]:
#Intancia do simulador
simulador=SimulatorLEA()
filepath="./dados_experimentais/Dados_BCSLEA_20210818.mat"
#Carregar dados experimentais
intervalo_horas=(0,7) #Janela de simulação em horas escolhida com base no histórico de dados
simulador.getLEAdata(filepath,intervalo_horas)


0
 File loaded ./dados_experimentais/Dados_BCSLEA_20210818.mat


In [16]:
fk,zc,pman,pres=simulador.BCS_EXP.get_exo_values() # Armazenamento dos valores das exógenas
x1,x2,x3=simulador.BCS_EXP.get_ss_values() # Armazenamento dos valores dos estados

maxtime = fk.shape[0]
nsim=maxtime

In [17]:
x1=np.reshape(x1,(maxtime,1))
x2=np.reshape(x2,(maxtime,1))
x3=np.reshape(x3,(maxtime,1))
fk=np.reshape(fk,(maxtime,1))
zc=np.reshape(zc,(maxtime,1))
pman=np.reshape(pman,(maxtime,1))
pres=np.reshape(pres,(maxtime,1))

In [21]:
x0,xc.shape

(array([[1.00000000e+05, 2.00000000e+05, 2.77777778e-05]]), (1, 3))

In [28]:
xc,x0=norm_values()
xc=xc.T;x0=x0.T
x1n=(x1-x0[0])/xc[0]
x2n=(x2-x0[1])/xc[1]
x3n=(x3-x0[2])/xc[2]
tempo=simulador.BCS_EXP.tempo_hora
x=np.concatenate((x1,x2,x3),axis=1)
print(len(x))
simulador.plotLEA.plot_states(x.T,tempo)


252


ValueError: Invalid arguments for input:x (only len<=2